In [1]:
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Sample Data
Hayat = [
"Olumlu düşünen, bütün olası dünyaların en iyisinde yaşadığımızı söyler ve olumsuz düşünen, bunun gerçek olmasından korkar.",
"Bu dünyaya gelmenin sadece tek bir yolu vardır, terk etmenin ise çok fazla yolu.",
"Birisinin ‘Hayat zor.’ diye yakındığını duyduğumda, her zaman, ‘Neye kıyasla?’ diye sormayı isterim.",
"Mutsuz olmamamız gerekir. Kimsenin hayatla bir kontratı yok.",
"Hayatını kazanırken yaşamasını bilmeyen bir adam, servetini kazanmadan öncesine göre daha fakirdir.",
"Yarının ne olacağını sormaktan vazgeç. Her gün, sana verilen bir hazinedir. Eline geçtikçe değerlendir.",
"Hayatınızı yaşamanın en iyi yolu, sizden sonra da kalıcı olacak şeyler için harcamaktır.",
"Biz Japonlar, küçük zevklerden hoşlanırız, israftan değil. Daha fazlasını karşılayabilirse de, insanın basit bir yaşam tarzı olması kanısındayım.",
"Benim sanatım ve mesleğim yaşamaktır.",
"Hayat, çikolata ile dolu bir kutu gibidir. Ne çıkacağını asla bilemezsiniz.",
"Hayat, büyük bir sürprizdir. Ölümün neden daha büyük bir sürpriz olması gerektiğini anlayamıyorum."
]

Sevgi = [
"Karım,  benim  ‘Seni seviyorum.’  dememi binlerce kez duydu ama, hiçbir zaman ‘Üzgünüm.’ dediğimi duymadı.",
"Büyüklerine saygı, küçüklerine şevkat göstermeyenler, bizden değildir.",
"Sevip de kaybetmek, sevmemiş olmaktan daha iyidir.",
"Sevgiyle düşünün, sevgiyle konuşun, sevgiyle davranın. Her ihtiyaç karşılanacaktır.",
"Sevgi, insanı birliğe, bencillik ise yalnızlığa götürür.",
"Sevdiği kadını ve sevdiği işi bulan bir erkek, yeryüzünde cenneti bulmuş demektir.",
"Kızgınlık, kırgınlık ya da acıya tutunmayın. Enerjinizi çalarlar ve sizi sevmekten alıkoyarlar.",
"Herkes temel bir şeyi unutuyor; insanlar sizi, siz onları sevmeden sevmeyecekler.",
"Sevgi, dünyadaki yaşamın aktığı nehirdir.",
"Sevmeden verebilirsiniz ama, vermeden sevemezsiniz."
]

Gayret = [
"Dileyin verilecektir; arayın bulacaksınız; kapıyı çalın size açılacaktır.",
"Damlayan su, mermeri, yürüyen de dağları deler.",
"Hazine, eziyet çekene gözükür.",
"Beklenen gün gelecekse, çekilen çile kutsaldır.",
"Yarınlar yorgun ve bezgin kimseler değil, rahatını terk edebilen gayretli insanlara aittir.",
"Emek olmadan, yemek olmaz.",
"Kötülüğün hakim olmaması için tek şart, iyilerin gayret göstermeleridir.",
"İnsan bir şeyi çok ciddi olarak arzu etmeye görsün; hiçbir şey erişilemeyecek kadar yükseklerde değildir.",
"Ümitsizlik ve korkaklık, çalışma azminden yoksun insanlara göredir.",
"Bir çığlık, bir çığ meydana getirir.",
"Kıramıyacağım zinciri, hiç olmazsa kemir.",
"Basit bir insanın elinden geleni yapması; bilgili ve zeki bir insanın tembelliğinden çok daha değerlidir.",
"Taşı delen, suyun kuvveti değil; dalgaların sürekliliğidir.",
"Damlayan su, taşı deler."
]

Idare = [
"Çok söyleyen değil, çok iş yapan yöneticiye muhtaçsınız.",
"Sevginin kurduğu devleti, adalet devam ettirir.",
"Her memleketin hakettiği bir hükümeti vardır.",
"Hükümetlerin en kötüsü, suçsuzu korkutandır.",
"Kendinizi yönetirken kafanızı, başkalarını yönetirken kalbinizi kullanın.",
"Evi ev eden avrat, yurdu şen eden devlet.",
"Mal cimrilerde, silah korkanlarda, idare de zayıflarda olursa, işler bozulur.",
"İnsanlar üç sınıftır: İdareciler, din alimleri ve halk. İdareciler bozulunca, geçim; alimler bozulunca, din; halk bozulunca da hak bozulur.",
"Kendini yönet, dünyayı yönetecek gücü bulabilirsin.",
"Kötü idarecilerin başarısı, halkın ıstırabıdır.",
"Bir memlekette ayaklar baş olursa, başlar ayaklar altında mahvolur."
]

Umut = [
"Başlangıçta fazla umut ederiz ama, sonrasında yeteri kadar değil.",
"Hayatta umutsuz durumlar yoktur,  sadece umutsuzluk besleyen insanlar vardır.",
"Şafaktan önce her yer karanlıktır.",
"Umut, gözle görülemeyeni görür, elle tutulamayanı hisseder ve imkansızı başarır.",
"Büyük umutlar, büyük adamlar yaratır.",
"Umut, fakirin ekmeğidir.",
"Kazanmayı umut etmeyen, daha baştan kaybetmiştir.",
"Umut etmeyen, hiçbir zaman umutsuzluğa kapılmaz.",
"Hiçbir kış, sonsuza kadar sürmez.",
"Umut, şüphe etmek daha kolayken, inanmaya devam etmektir.",
"Umut, mutluluktan alınmış bir miktar borçtur."
]

# Combine all documents
all_documents = Hayat + Sevgi + Gayret + Idare + Umut

# Preprocess text
stop_words = set(stopwords.words('turkish'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return words

def preprocess_documents(documents):
    return [' '.join(preprocess(doc)) for doc in documents]

all_processed_docs = preprocess_documents(all_documents)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BatuhanYILMAZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BatuhanYILMAZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load pre-trained Word2Vec model
word_vectors = KeyedVectors.load_word2vec_format('trmodel', binary=True)

# Function to expand query using Word2Vec
def expand_query(query, top_n=2):
    processed_query = preprocess(query)
    expanded_query = processed_query.copy()
    for word in processed_query:
        if word in word_vectors.key_to_index:
            similar_words = word_vectors.most_similar(positive=[word], topn=top_n)
            expanded_query.extend([word[0] for word in similar_words])
    return ' '.join(expanded_query)


In [3]:
# TF-IDF Vectorizer
cv = CountVectorizer()
word_count_vector = cv.fit_transform(all_processed_docs)
tfidf_transformer = TfidfTransformer(smooth_idf=True, norm='l2', use_idf=True)
tfidf_transformer.fit(word_count_vector)
tf_idf_vector = tfidf_transformer.transform(word_count_vector)
W = tf_idf_vector.toarray()


In [8]:
import numpy as np
# Function to respond to query using TF-IDF + Word2Vec only
def respond(query, n_best=5):
    # Expand the query using Word2Vec
    expanded_query = expand_query(query)
    print(f"Expanded query: {expanded_query}\n")
    
    # Convert the expanded query to TF-IDF vector
    expanded_query_vec = cv.transform([expanded_query])
    tf_idf_query = tfidf_transformer.transform(expanded_query_vec)
    
    # Compute cosine similarity for TF-IDF
    tfidf_similarities = cosine_similarity(tf_idf_query, W)
    
    # Rank documents based on the TF-IDF similarities
    sorted_doc_indices = np.argsort(tfidf_similarities[0])[::-1]
    
    # Print the sorted documents and their similarity scores
    for idx in sorted_doc_indices[:n_best]:
        print(f"{all_processed_docs[idx]} with similarity score: {tfidf_similarities[0][idx]:.4f}\n")

# Make a search
query = 'hayat gerçek çünkü güzel'
respond(query)


Expanded query: hayat gerçek güzel yaşam yaşantı asıl aslında guzel eğlenceli

japonlar küçük zevklerden hoşlanırız israftan değil fazlasını karşılayabilirse insanın basit bir yaşam tarzı olması kanısındayım with similarity score: 0.1666

hayat çikolata dolu bir kutu gibidir çıkacağını asla bilemezsiniz with similarity score: 0.1522

olumlu düşünen bütün olası dünyaların iyisinde yaşadığımızı söyler olumsuz düşünen bunun gerçek olmasından korkar with similarity score: 0.1519

birisinin hayat zor yakındığını duyduğumda zaman neye kıyasla sormayı isterim with similarity score: 0.1402

hayat büyük bir sürprizdir ölümün büyük bir sürpriz olması gerektiğini anlayamıyorum with similarity score: 0.1302



In [9]:
query = 'adalet mülkün temelidir'
respond(query)

Expanded query: adalet mülkün temelidir fazilet kalkınma itibarla hususta doğasını anlaşılmasında

sevginin kurduğu devleti adalet devam ettirir with similarity score: 0.4144

umut mutluluktan alınmış bir miktar borçtur with similarity score: 0.0000

sevgiyle düşünün sevgiyle konuşun sevgiyle davranın ihtiyaç karşılanacaktır with similarity score: 0.0000

yarınlar yorgun bezgin kimseler değil rahatını terk edebilen gayretli insanlara aittir with similarity score: 0.0000

beklenen gün gelecekse çekilen çile kutsaldır with similarity score: 0.0000

